In [1]:
from PIL import Image
import os, glob
import numpy as np
import random, math

In [2]:
root_dir = "FCBarcelona"

In [3]:
categories =  ["メッシ","スアレス","グリーズマン","フレンキー・デヨング","ラキティッチ","アルトゥール","ウスマヌデンベレ","セルヒオブスケツ","ウンティティ","テアシュテーゲン","ビダル","ピケ","セルジロベルト","ラングレ"]

In [4]:
X = []
Y = []

In [5]:
def make_sm(files):
    global X, Y
    X = []
    Y = []
    for cat, fname in files:
        add_sm(cat, fname)
    return np.array(X), np.array(Y)

In [6]:
def add_sm(cat, fname):
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((150,150))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

allfiles = []

In [7]:
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

In [8]:
random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.8)
train = allfiles[0:th]
test  = allfiles[th:]
X_train, y_train = make_sm(train)
X_test, y_test = make_sm(test)
xy = (X_train, X_test, y_train, y_test)
#データを保存する（データの名前を「tea_data.npy」としている）
np.save("player_data.npy", xy)

In [9]:
display(allfiles)

[]

In [10]:
from keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(10,activation="sigmoid")) #分類先の種類分設定

#モデル構成の確認
model.summary()

C:\Users\masaharu\Anaconda3\envs\kaggle_first\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
W1205 18:21:19.858529 83996 deprecation_wrapper.py:119] From C:\Users\masaharu\Anaconda3\envs\kaggle_first\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1205 18:21:19.926171 83996 deprecation_wrapper.py:119] From C:\Users\masaharu\Anaconda3\envs\kaggle_first\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1205 18:21:19.945937 83996 deprecation_wrapper.py:119] From C:\Users\masaharu\Anaconda3\envs\kaggle_first\lib\site-packages\keras\b

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [11]:
from keras import optimizers

model.compile(loss="binary_crossentropy",
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=["acc"])

W1205 18:21:20.483724 83996 deprecation_wrapper.py:119] From C:\Users\masaharu\Anaconda3\envs\kaggle_first\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1205 18:21:20.508049 83996 deprecation_wrapper.py:119] From C:\Users\masaharu\Anaconda3\envs\kaggle_first\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1205 18:21:20.523464 83996 deprecation.py:323] From C:\Users\masaharu\Anaconda3\envs\kaggle_first\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
from keras.utils import np_utils
import numpy as np

categories =  ["メッシ","スアレス","グリーズマン","フレンキー・デヨング","ラキティッチ","アルトゥール","ウスマヌデンベレ","セルヒオブスケツ","ウンティティ","テアシュテーゲン","ビダル","ピケ","セルジロベルト","ラングレ"]
nb_classes = len(categories)

X_train, X_test, y_train, y_test = np.load("player_data.npy")

#データの正規化
X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float")  / 255

#kerasで扱えるようにcategoriesをベクトルに変換
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test  = np_utils.to_categorical(y_test, nb_classes)

In [13]:
model = model.fit(X_train,
                  y_train,
                  epochs=15,
                  batch_size=6,
                  validation_data=(X_test,y_test))

ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (0, 1)